In [1]:
import pandas as pd
import numpy as np

from pathlib import Path
# import seaborn as sns
from tqdm import tqdm

# import matplotlib.pyplot as plt
# import matplotlib
# %matplotlib inline


import backtrader as bt
import backtrader.indicators as btind

import datetime



In [2]:
# backtrader is compatible with matplotliv version 3.2.0
# pip install --upgrade matplotlib==3.2.0


### Load Data

In [3]:
db_path = Path("../../Data/Crypto/crypto.h5")
db_path

WindowsPath('../../Data/Crypto/crypto.h5')

In [4]:
df_pv = pd.read_hdf(db_path, f"Min60")

df_pv = df_pv.reset_index()
df_pv = df_pv.set_index('time')

df_pv.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 21600 entries, 2022-01-01 01:00:00 to 2022-06-30 00:00:00
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      21600 non-null  object 
 1   low     21600 non-null  float64
 2   high    21600 non-null  float64
 3   open    21600 non-null  float64
 4   close   21600 non-null  float64
 5   volume  21600 non-null  float64
dtypes: float64(5), object(1)
memory usage: 1.2+ MB


In [5]:
df_pv['id'].unique()

array(['BTC-USD', 'ETH-USD', 'SOL-USD', 'AVAX-USD', 'ADA-USD'],
      dtype=object)

### Preparation Func

In [6]:
ticker_list = df_pv['id'].unique()


## in case add self-defined features
class PandasData_more(bt.feeds.PandasData):
    lines = ('pe', ) # add self-defined data

    params=(
        ('pe', -1),
           )


def feedData(cerebro, ticker_list):

    for ticker in ticker_list:
        # process the data into bt format
        data = pd.DataFrame(index=df_pv.index.unique())
        df = df_pv.query(f"id=='{ticker}'")[['open','high','low','close','volume']]
        data_ = pd.merge(data, df, left_index=True, right_index=True, how='left')

        # process nan
        data_.loc[:,'volume'] = data_.loc[:, 'volume'].fillna(0)
        data_.loc[:,['open','high','low','close']] = data_.loc[:,['open','high','low','close']].fillna(method='pad')
        data_.loc[:,['open','high','low','close']] = data_.loc[:,['open','high','low','close']].fillna(0)

        # self-defined data
        data_['pe'] = 1
        
        # feed data
        datafeed = PandasData_more(dataname=data_,
                                    fromdate=data_.index[0], todate=data_.index[-1]
                                    )
        cerebro.adddata(datafeed, name=ticker) # match ticker
        print(f"{ticker} Done !")

##### Unit Test

In [7]:
class TestStrategy(bt.Strategy):
    def __init__(self):

        print("Ticker:", self.datas[0]._name) 
        print("Fields: ", self.datas[0].lines.getlinealiases())
        print("Close: ", self.datas[0].lines.close[0])
        print("Low: ", self.datas[0].lines.low[0])
        print("Volume: ", self.datas[0].lines.volume[0])
        print("Datetime: ", self.datas[0].lines.datetime[0])

    
    def next(self):
        for ticker in self.getdatanames():
            if ticker == 'ETH-USD':
                data = self.getdatabyname(ticker)
            
                print("Datetime: ", bt.num2date(data.lines[6][0]), "| Close: ", data.lines.close[0])
                # print(bt.num2date(self.datas[0].lines[6][0]), self.datas[0].lines.close[0])

cerebro_test = bt.Cerebro()
ticker_list = ['BTC-USD', 'ETH-USD']
# datafeed = bt.feeds.PandasData(dataname=df_pv.query(f"id=='{ticker}'")[['open','high','low','close','volume']])
# cerebro_test.adddata(datafeed, name=ticker)

feedData(cerebro_test, ticker_list)

cerebro_test.addstrategy(TestStrategy)
result = cerebro_test.run()

BTC-USD Done !
ETH-USD Done !
Ticker: BTC-USD
Fields:  ('close', 'low', 'high', 'open', 'volume', 'openinterest', 'datetime', 'pe')
Close:  20012.78
Low:  19980.28
Volume:  788.58643047
Datetime:  738336.0
Datetime:  2022-01-01 01:00:00 | Close:  3724.22
Datetime:  2022-01-01 02:00:00 | Close:  3727.94
Datetime:  2022-01-01 03:00:00 | Close:  3722.96
Datetime:  2022-01-01 04:00:00 | Close:  3706.87
Datetime:  2022-01-01 05:00:00 | Close:  3735.0
Datetime:  2022-01-01 06:00:00 | Close:  3710.33
Datetime:  2022-01-01 07:00:00 | Close:  3714.27
Datetime:  2022-01-01 08:00:00 | Close:  3720.88
Datetime:  2022-01-01 09:00:00 | Close:  3718.44
Datetime:  2022-01-01 10:00:00 | Close:  3691.49
Datetime:  2022-01-01 11:00:00 | Close:  3693.24
Datetime:  2022-01-01 12:00:00 | Close:  3716.94
Datetime:  2022-01-01 13:00:00 | Close:  3710.46
Datetime:  2022-01-01 14:00:00 | Close:  3698.51
Datetime:  2022-01-01 15:00:00 | Close:  3726.65
Datetime:  2022-01-01 16:00:00 | Close:  3728.81
Datetime:  

In [8]:
df_pv.query(f"id == '{ticker_list[1]}'")

,id,low,high,open,close,volume
time,,,,,,
2022-01-01 01:00:00,ETH-USD,3709.15,3748.11,3722.79,3724.22,9223.416393
2022-01-01 02:00:00,ETH-USD,3722.01,3737.38,3724.81,3727.94,5135.417450
2022-01-01 03:00:00,ETH-USD,3721.99,3736.08,3728.01,3722.96,3711.649049
2022-01-01 04:00:00,ETH-USD,3705.09,3727.34,3723.01,3706.87,6879.855088
2022-01-01 05:00:00,ETH-USD,3702.43,3763.83,3706.86,3735.00,5943.013057
...,...,...,...,...,...,...
2022-06-29 20:00:00,ETH-USD,1103.65,1120.99,1116.39,1108.00,14324.687134
2022-06-29 21:00:00,ETH-USD,1105.78,1117.35,1107.93,1112.55,16219.461009
2022-06-29 22:00:00,ETH-USD,1102.86,1118.49,1112.56,1107.79,8415.103830


### Inititalize cerebro 

In [21]:
cerebro = bt.Cerebro(optdatas=True, optreturn=True)

### Feed Data

In [22]:
ticker_list = df_pv['id'].unique()
ticker_list = ticker_list[1:3]
ticker_list

array(['ETH-USD', 'SOL-USD'], dtype=object)

In [23]:
feedData(cerebro, ticker_list)

ETH-USD Done !
SOL-USD Done !


### Config

In [24]:
# initial cash
cerebro.broker.setcash(100000.0)
# commission
cerebro.broker.setcommission(commission=0.0003)
# slippage
cerebro.broker.set_slippage_perc(perc=0.0001)

### Strategy

In [25]:
class DefaultStrategy(bt.Strategy):
    params = (('period1', 12),
        ('period2', 48),
        ('period3', 120)
    )

    # trading logs
    def log(self, txt, dt=None):           
        dt = dt or self.datas[0].datetime.datetime(0)
        print('%s, %s' % (dt.isoformat(), txt))

    
    
    def notify_order(self, order):
        # free cash
        total_cash = self.broker.getcash()
        # ticker's existing size
        hold_size = self.broker.getposition(order.data).size

        total_hold = 0
        for ticker in self.getdatanames(): 
            total_hold += self.broker.getposition(self.getdatabyname(ticker)).price * self.broker.getposition(self.getdatabyname(ticker)).size
            
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed, order.Canceled, order.Margin]:
            if order.isbuy():
                self.log(
                        'BUY EXECUTED, ref:%.0f, Price: %.2f, Cost: %.2f, Comm %.2f, Size: %.2f, Stock: %s, HoldSize: %.2f' %
                        (order.ref, 
                        order.executed.price, 
                        order.executed.value, 
                        order.executed.comm, 
                        order.executed.size, 
                        order.data._name,
                        hold_size
                        ))
                self.log('Total_Cash: %.2f, Total_Hold: %.2f, Total_Value: %.2f' %
                    (total_cash, total_hold, total_cash+total_hold)
                    )
            else: # Sell
                self.log('SELL EXECUTED, ref:%.0f, Price: %.2f, Cost: %.2f, Comm %.2f, Size: %.2f, Stock: %s, HoldSize: %.2f' %
                            (order.ref,
                            order.executed.price,
                            order.executed.value,
                            order.executed.comm,
                            order.executed.size,
                            order.data._name,
                            hold_size
                            ))
                self.log('Total_Cash: %.2f, Total_Hold: %.2f, Total_Value: %.2f' %
                    (total_cash, total_hold, total_cash+total_hold)
                    )

    # def start(self):
    #     self.mystats = csv.writer(open("mystats.csv", "w"))
    #     self.mystats.writerow(['datetime',
    #                            'drawdown', 'maxdrawdown',
    #                            'timereturn',
    #                            'value', 'cash'])

    def __init__(self):
        self.order = None
        self.status = 0


        for ticker in self.getdatanames():
            data = self.getdatabyname(ticker)
            # construct MovingAverage
            data.lines.sma1 = bt.ind.SMA(data.lines.close, period = self.p.period1)
            data.lines.sma2 = bt.ind.SMA(data.lines.close, period = self.p.period2)
            data.lines.sma3 = bt.ind.SMA(data.lines.close, period = self.p.period3)



    def next(self):

        # dt = self.datas[0].datetime.date(0)
        # if the order is not completed, do not generate new one
        if self.order:
            return  

        for ticker in self.getdatanames():
            # getdatanames()
            data = self.getdatabyname(ticker)

            hold_price = self.broker.getposition(data).price
            hold_size = self.broker.getposition(data).size
            cash = self.broker.getcash()

            buy_size = int(0.2 * cash / data.lines.close[0])

            sell_size = hold_size


            # cross above
            bug_sig = data.lines.sma2[0] >= data.lines.sma3[0] and data.lines.sma2[-1] < data.lines.sma3[-1]
            # cross below
            sell_sig = data.lines.sma1[0] <= data.lines.sma2[0] and data.lines.sma1[-1] > data.lines.sma2[-1]

            if bug_sig:
                # market order
                self.buy(data = data, size = buy_size, exectype=bt.Order.Market)
                # limit order
                # self.buy(exectype=bt.Order.Limit, price=price, valid=valid) 
            if sell_sig:
                self.sell(data = data, size = sell_size, exectype=bt.Order.Market)        
        

In [26]:
# cerebro.optstrategy(DefaultStrategy, period1=range(20, 25, 5), period2=range(60, 70, 10))
cerebro.addstrategy(DefaultStrategy)

# add observer
# cerebro.addobserver(bt.observers.Broker)
# cerebro.addobserver(bt.observers.Trades)
# cerebro.addobserver(bt.observers.BuySell)
# cerebro.addobserver(bt.observers.DrawDown)
cerebro.addobserver(bt.observers.TimeReturn)

# add analyzer
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')
cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='_TimeReturn')
cerebro.addanalyzer(bt.analyzers.Returns, _name='_Returns', tann=252)

cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='_SharpeRatio', timeframe=bt.TimeFrame.Days, annualize=True, riskfreerate=0) 


cerebro.addanalyzer(bt.analyzers.DrawDown, _name='_DrawDown')

results = cerebro.run()
# results = cerebro.run(maxcpus=2)


2022-01-12T14:00:00, BUY EXECUTED, ref:68, Price: 3372.24, Cost: 16861.19, Comm 5.06, Size: 5.00, Stock: ETH-USD, HoldSize: 5.00
2022-01-12T14:00:00, Total_Cash: 83133.76, Total_Hold: 16861.19, Total_Value: 99994.94
2022-01-12T22:00:00, BUY EXECUTED, ref:69, Price: 152.03, Cost: 16570.75, Comm 4.97, Size: 109.00, Stock: SOL-USD, HoldSize: 109.00
2022-01-12T22:00:00, Total_Cash: 66558.04, Total_Hold: 33431.93, Total_Value: 99989.97
2022-01-13T23:00:00, SELL EXECUTED, ref:70, Price: 3247.44, Cost: 16861.19, Comm 4.87, Size: -5.00, Stock: ETH-USD, HoldSize: 0.00
2022-01-13T23:00:00, Total_Cash: 82790.34, Total_Hold: 16570.75, Total_Value: 99361.09
2022-01-14T04:00:00, SELL EXECUTED, ref:71, Price: 147.57, Cost: 16570.75, Comm 4.83, Size: -109.00, Stock: SOL-USD, HoldSize: 0.00
2022-01-14T04:00:00, Total_Cash: 98870.13, Total_Hold: 0.00, Total_Value: 98870.13
2022-01-26T18:00:00, BUY EXECUTED, ref:72, Price: 2597.64, Cost: 18183.48, Comm 5.46, Size: 7.00, Stock: ETH-USD, HoldSize: 7.00
202

In [27]:
result

In [28]:
def get_my_analyzer(result):
    analyzer = {}

    analyzer['period1'] = result.params.period1
    analyzer['period2'] = result.params.period2
    analyzer['period3'] = result.params.period3

    analyzer['annual_return'] = result.analyzers._Returns.get_analysis()['rnorm']
    analyzer['annual_return(%)'] = result.analyzers._Returns.get_analysis()['rnorm100']

    analyzer['drawdown'] = result.analyzers._DrawDown.get_analysis()['max']['drawdown'] * (-1)

    analyzer['sharperatio'] = result.analyzers._SharpeRatio.get_analysis()['sharperatio']
    
    return analyzer

ret = []
for result in results:
    ret.append(get_my_analyzer(result))
    
pd.DataFrame(ret)

,period1,period2,period3,annual_return,annual_return(%),drawdown,sharperatio
0,12,48,120,-0.154498,-15.449775,-12.76026,-1.801412


In [29]:
results[0].analyzers._DrawDown.get_analysis()
results[0].analyzers._SharpeRatio.get_analysis()
# result[0].analyzers._TimeReturn.get_analysis()

OrderedDict([('sharperatio', -1.801411912229653)])

In [30]:
cerebro.plot(iplot=False)
# cerebro.plot()

KeyboardInterrupt: 

In [19]:
%matplotlib inline
cerebro.plot()

<IPython.core.display.Javascript object>

[[<Figure size 640x480 with 8 Axes>]]